In [1]:
import pandas as pd
import sys

#csv에서 news content를 txt로 변환

csv=pd.read_csv('../Data/201903.csv',encoding='utf-8-sig')

text=csv['text']
print(text)

with open('News_Contents.txt','w',encoding='utf-8-sig') as file:
    for item in text:
        file.write("%s\n" % item)


0        美대표 “이행 점검 장·차관급 회담 年 6회 中, 합의 안 지킬 땐 관세 폭탄 즉시...
1        법원 “여성 2명·자녀 6명 송환 의무 없어” 벨기에 “10세 이하 어린이는 데려올...
2        “국제 원조 받아들이고 대선 다시 치러야”“美 인도주의적 쇼 원조물품 일단 회수를”...
3        전과자 등 총기 소유 막기 위해 의무화 상원 반대·트럼프 거부에 통과 불투명민주당이...
4        성능 실험서 배수 등 중대 결함 발견  도널드 트럼프 미국 대통령이 국경장벽 건설을...
                               ...                        
22383    청문회서 결정적 흠결 드러나 / 잇단 부실 검증 책임 물어야 / 인사기준 강화가 우...
22384    자유한국당이 축구장 선거유세 논란에 대해 “선거관리위원회로부터 유권해석을 받은 것”...
22385    일회용 비닐봉투 금지된 대형마트일회용 비닐봉투 사용이 금지된 1일 서울의 한 대형마...
22386    현대모비스는 일산 킨텍스에서 열린 ‘2019 서울모터쇼’를 통해 연말까지 글로벌 최...
22387    문재인 대통령이 31일 조동호 과학기술정보통신부 장관 후보자에 대한 지명을 철회했다...
Name: text, Length: 22388, dtype: object


In [2]:
import pandas as pd

df = pd.read_csv('../Data/201903.csv', encoding='utf-8-sig')

In [3]:
from kiwipiepy import Kiwi
import pandas as pd
import math

kiwi = Kiwi()
kiwi.load_user_dictionary('new_wordlist2.txt')
kiwi.prepare()

stopword = []
def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8-sig")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if wordlist[1].startswith('N'):
            stopword.append(wordlist[0])
    stopwords = set(stopword)
    files.close()
    return stopword

def npmi(a, b, n, z):
    return (math.log((n*z)/(a*b))) / (math.log(n/z))

# def npmi3(a, b, c, n, z):
#     a = math.log((n*n*z)/(a*b*c))
#     b = 2*math.log(n/z)
#     return a / b

In [4]:
import tomotopy as tp

def extract_topic(doc):
    model = tp.LDAModel(k=1, min_cf=2, min_df=1, tw=tp.TermWeight.PMI)
    for line in doc:
        if(len(line) == 0):
            continue
        model.add_doc(line) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    for j in range(0,100):
        model.train(10) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    s = []
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
#         print(k,res)
        for w, p in res:
            s.append(w)

        # s = ', '.join(w for w, p in res)
    # print(s)
    return ', '.join(s)

In [5]:
from kss import split_sentences
from collections import Counter
import pprint as pp
import re

rm_stopword('한국어불용어100.txt')
docs = []
merge_token = []
bi_gram = {}
LDA_Topics = []
i = 0
for news in open('News_Contents.txt', encoding='utf-8'):
    doc = []
    context = []
    com = []
    lines = split_sentences(news)

    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word.strip() for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
#         context += content
        doc.append(content)
    LDA_Topics.append(extract_topic(doc))
#     #2 token npmi
#     for line in doc:
#         com += list(zip(line, line[1:]))
#     nmi = {}
#     n = len(context)
#     cnt = Counter(context)
#     z = Counter(com)
#     for pair in com:
#         pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
#         if -1 < pmi < 1:
#             nmi[pair] = pmi
#     result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
#     tmp = [key[0]+" "+key[1] for key, item in result if item > 0.89]

    #2 token N_GRAM 연결
    # tmp = []
    # for line in doc:
    #     tmp += list(zip(line, line[1:]))
#     for word in tmp:
#         if word in bi_gram:
#             bi_gram[word] += 1
#         else:
#             bi_gram[word] = 1
    
    # bi_gram = sorted(bi_gram.items(), key = lambda item : item[1], reverse=True)
    # merge_token = {key for key, item in bi_gram if item > 2}
    # print(merge_token)

    
    # 토큰을 합치는 내용
    # for token in merge_token:
    #     for content in doc:
    #         for i in range(len(content)-1):
    #             cmp_str = '_'.join(content[i:i+2])
    #             if cmp_str == token:
    #                 content.pop(i)
    #                 content.pop(i)
    #                 content.insert(i, token)
    docs.append(doc)
    # print(len(docs),"===================================")
    if len(docs) % 1000 == 0:
        print(len(docs))

# bi_gram = sorted(bi_gram.items(), key = lambda item : item[1], reverse=True)

# with open('pmi_token.txt','w',encoding='utf-8-sig') as file:
#     for key, value in bi_gram:
#         file.write("%s" % (key+' , '+str(value)+"\n"))
#         if value < 20:
#             break


df['N_list'] = docs
df['topics'] = LDA_Topics
df.to_csv('../Data/2019/201903_LDA.csv', encoding='utf-8-sig', index = False)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [163]:
i = 0
s_list = []

for doc in docs:
    if(i%100 == 99):
        break
    print(doc)
    break

[['문희상', '국회', '의장', '새해', '대한민국 임시정부', '임시', '정원', '해', '역경', '시련', '질곡', '영광', '매듭', '역사', '시작', '기념비', '한 해'], ['문 의장', '신년사', '대한민국', '기적', '변화', '세계', '주목', '북한 비핵화', '한반도 평화', '프로세스', '실현', '중대', '기로'], ['문 의장', '차 산업', '혁명', '거대한', '물결', '민생', '경제', '성패', '가늠', '수십', '퇴보', '절체절명'], ['문 의장', '대', '국회', '개헌', '선거', '제도', '정치', '개혁', '실천', '여부', '판가름', '마지막', '해', '선거', '제도', '개혁', '강조'], ['문 의장', '국민', '통합', '한반도', '평화', '협치', '신뢰', '세계', '미래', '도약', '대한민국', '원년', '대한민국', '국회', '혼신', '힘', '다짐'], ['다음', '신년사', '전문', '김태규', '기자']]
